[도서]  [데이터실무분석 with파이썬 예제](https://wikibook.co.kr/playwithdata/)

# 4.1 인스타그램 크롤링 

### (1) selenium 설치 
   
    + pip install selenium
    
    + 웹드라이버 다운로드
        http://chromedriver.chromium.org/downloads
        chromedirver_win32.zip 파일 다운로드 받고 압축풀기

        설치된 크롬 버전과 맞는 버전을 다운로드 해야 한다.
        
        
---        
[참고] 사이트 변경으로 인해 데이타를 못 가져올 수도 있다        

### 4.1.2 인스타그램 검색 결과 URL을 만들어서 접속하기

In [9]:
# 예제 4-1 인스타그램 검색결과 URL을 만드는 함수
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
    

In [10]:
# !pip install selenium

In [11]:
webdriverpath = 'D:/python-workspace/PythonBasic/webdriver/chromedriver.exe' 

In [12]:
# 예제 4-2 selenium으로 URL 접속하기
from selenium import webdriver

# 본인의 webdriver 설치 경로 맞추기
#driver = webdriver.Chrome("D:/MyClass/Python/eAnalysisExample/2_제주맛집_크롤링_지도/webdriver/chromedriver.exe")   
driver = webdriver.Chrome(webdriverpath)   

word = "제주도맛집"
url = insta_searching(word)
driver.get(url)


### 4.1.3 첫 번째 게시글 열기 

In [13]:
# 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    
    driver.implicitly_wait(10) # 로딩 속도 때문에 클릭이벤트를 못하기에 추가 ( 상황에 따라 실행되었다 안되었다 함 )
    
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

### 4.1.4 게시글 정보 가져오기 

In [14]:
# 예제 4-4 게시글 정보 가져오기
import re

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    print('get_content 확인', date)
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data


### 4.1.5 다음 게시글 열기

In [15]:
# 예제 4-5 다음 게시글 열기
def move_next(driver):

    driver.implicitly_wait(10) # 로딩 속도 때문에 클릭이벤트를 못하기에 추가 ( 상황에 따라 실행되었다 안되었다 함 )
    
    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)


### 4.1.6 여러 게시글 정보 수집하기

In [22]:

import time
#------------------------------------------------------------------------------------
def readp():
    f = open("D:/python-workspace/PythonBasic/webdriver/a.txt")
    a = f.readline()
    return a
def login(driver):
    driver.find_elements_by_css_selector('label.f0n8F > input')[0].send_keys('010-3303-1995')
    driver.find_elements_by_css_selector('label.f0n8F > input')[1].send_keys(readp())
    driver.find_elements_by_css_selector('button.sqdOP > div.Igw0E')[0].click()
    time.sleep(3)
    driver.find_elements_by_css_selector('button.sqdOP')[0].click()
    
# ① 크롬 브라우저 열기
driver = webdriver.Chrome(webdriverpath) # 테스트 하고 
# ② 인스타그램 검색페이지 URL 만들기
word = "제주도맛집"    #검색어
url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(3)
#------------------------------------------------------------------------------------
select_first(driver)
login(driver)
select_first(driver)


    
    

a


In [26]:
# 예제 4-6 인스타그램 크롤링
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# ① 크롬 브라우저 열기
#driver = webdriver.Chrome('D:/MyClass/Python/eAnalysisExample/2_제주맛집_크롤링_지도/webdriver/chromedriver.exe') 
driver = webdriver.Chrome(webdriverpath)

# ② 인스타그램 검색페이지 URL 만들기
word = "제주도맛집"    #검색어
url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(3)

# ④ 첫 번째+ 게시글 열기
select_first(driver)
login(driver)
select_first(driver)
# ⑤ 비어있는 변수(results)만들기
results = [ ]


# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 5      # 크롤링할 게시글 수 : 우선은 5개 하고 실제적으로 50개
for i in range(target):
    data = get_content(driver)    # 게시글 정보 가져오기
    results.append(data)
    move_next(driver)

print(results[:2])

a
get_content 확인 2020-07-05
get_content 확인 2020-07-06
get_content 확인 2020-07-05
get_content 확인 2020-07-06
get_content 확인 2020-07-06
[['올여름은 제주도로 고고~제주갈치왕은 필수😍-여기갈치 요리가 최고로 맛있엉👍-제주산 갈치라 싱싱하고 살도 토실토실~해산물 폭탄으로 들어간 해물탕까지~살살 녹는 갈치요리 찐찐찐~ 맛집 갈치왕!🐟-여기 사장님의 손맛이 끝내주셔서밥 두공기는 기본으로 클리어~통갈치 해물찜과 술은 환상의 짝꿍~💏-🦄제주 풍경 보고~ 해산물, 갈치 요리 먹방하러 갈치왕으로~-@kingfish1190- 업체명 : 갈치왕 본점-주소 : 제주 서귀포시 일주서로 1146-전화 : 064-739-9333-#제주도맛집 #갈치왕#제주도맛집베스트 #제주도맛집추천#제주도맛집투어 #제주도핫플 #갈치맛집 #제주도갈치맛집 #제주갈치 #제주도갈치 #갈치구이맛집 #제주갈치구이 #제주갈치조림 #서귀포맛집추천 #제주가볼만한곳 #조림맛집 #제주도갈치맛집#좋반#좋아요반사#좋아요#좋반테러', '2020-07-05', 0, '', ['#제주도맛집', '#갈치왕', '#제주도맛집베스트', '#제주도맛집추천', '#제주도맛집투어', '#제주도핫플', '#갈치맛집', '#제주도갈치맛집', '#제주갈치', '#제주도갈치', '#갈치구이맛집', '#제주갈치구이', '#제주갈치조림', '#서귀포맛집추천', '#제주가볼만한곳', '#조림맛집', '#제주도갈치맛집', '#좋반', '#좋아요반사', '#좋아요', '#좋반테러']], ['애월카페 망고레이.새콤달콤 맛있는 망고와 함께 즐기는 제주바다망고레이에서 달콤한 망고와 함께즐거운 제주여행하세요~!...🍹망고레이(하귀본점)위치: 하귀9길 34, 애월해안도로 입구-영업시간: 10:00~18:00.🍹망고레이(함덕직영점)생망고와 수제코코넛아이스크림을 맛 볼 수 있는 곳!망고레이에서 다양하고 맛있는 디저트 만나보세요..#제주도맛집#제주여행#제주디저트카페 #제

In [ ]:
# 예제 4-7 크롤링 결과 저장하기

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./files/3_1_crawling_jejudoMatJip.xlsx')


### 4.1.8 여러 엑셀 파일의 중복을 제거한 후 통합 저장

인스타그램에서 여러 개의 키워드로 검색하거나 어러 번에 걸쳐 데이터를 수집하는 경우가 많다.

동일한 게시물을 중복으로 수집되는 경우 중복을 제거해야 한다.

In [ ]:
# 예제 4-8 여러 개의 저장파일 통합하기
jeju_insta_df = pd.DataFrame( [ ] )

folder = './files/'

# f_list = ['3_1_crawling_jejudoMatJip.xlsx', '3_1_crawling_jejudoGwanGwang.xlsx', '3_1_crawling_jejuMatJip.xlsx', '3_1_crawling_jejuYeoHang.xlsx']
f_list = ['3_1_crawling_jejuYeoHang.xlsx', '3_1_crawling_jejudoGwanGwang.xlsx','3_1_crawling_jejuYeoHang.xlsx','3_1_crawling_jejuMatJip.xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns =['content','data','like','place','tags']

# ???? jeju_insta_df.columns  왜 7개 이지???

In [ ]:
# 예제 4-9 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)
jeju_insta_df.to_excel('./files/3_1_crawling_raw.xlsx', index = False)